In [1]:
import nb_setup 
import importlib
importlib.reload(nb_setup)
nb_setup.init()

Added project root to sys.path: /Users/falcon/Developments/act/backtesting-playground/trading
Changed current working directory to: /Users/falcon/Developments/act/backtesting-playground/trading
Initialized project with base directory: /Users/falcon/Developments/act/backtesting-playground/trading


In [2]:
import models.market as market
import importlib
importlib.reload(market)

market = market.Market.from_json_file()
df = market.to_df()
top_60_coins_df = df.sort_values(by='dayNtlVlm', ascending=False).head(60)
top_60_coins_df = top_60_coins_df.reset_index(drop=True)
top_60_coins = top_60_coins_df.get('name')

In [3]:
from constants.coin_names import CoinNames
from settings import DATA_DIR
import pandas as pd
import os
dfs = {}
no_data = []
# for coin in CoinNames.__members__:
for coin in top_60_coins:
    p = DATA_DIR / f'{coin}_4h_380.csv'
    if os.path.exists(p):
        df = pd.read_csv(p, index_col=0, parse_dates=True)
        dfs[coin] = df
    else:
        no_data.append(coin)
else:
    os.sys.stderr.write(f'No data for {no_data}')

/var/folders/6w/7b7qhtd56jz545z6zwv6zc800000gn/T/ipykernel_34329/3002233825.py:11: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(p, index_col=0, parse_dates=True)
No data for []

In [4]:
dfs["SOL"].iloc[-1, :]["tic"]

1714003200000

In [5]:
from collections import Counter


def get_majority_shape(dfs):
    # Get the shapes of all DataFrames
    shapes = [df.shape for df in dfs.values()]
    
    # Count occurrences of each shape
    shape_counts = Counter(shapes)
    
    # Display votes for each shape
    for shape, count in shape_counts.items():
        print(f"Shape: {shape}, Votes: {count}")
    
    # Find the most common shape
    majority_shape, majority_votes = shape_counts.most_common(1)[0]
    
    return majority_shape

def get_majority_start_end(dfs):
    # Get the shapes of all DataFrames
    start_end = [(df.iloc[0, :]["tic"], df.iloc[-1, :]["tic"] ) for df in dfs.values()]
    
    # Count occurrences of each shape
    start_end_counts = Counter(start_end)
    
    # Display votes for each shape
    for se, count in start_end_counts.items():
        print(f"Start End: {se}, Votes: {count}")
    
    # Find the most common shape
    majority_se, majority_votes = start_end_counts.most_common(1)[0]
    
    return majority_se

major_shape = get_majority_shape(dfs)
major_se = get_majority_start_end(dfs)

Shape: (381, 12), Votes: 56
Shape: (136, 12), Votes: 1
Shape: (243, 12), Votes: 1
Shape: (99, 12), Votes: 1
Shape: (10, 12), Votes: 1
Start End: (1708531200000, 1714003200000), Votes: 56
Start End: (1712059200000, 1714003200000), Votes: 1
Start End: (1710518400000, 1714003200000), Votes: 1
Start End: (1712592000000, 1714003200000), Votes: 1
Start End: (1713873600000, 1714003200000), Votes: 1


In [6]:
def filter_dfs(dfs, shape, start_end):
    valid_dfs = {}
    for coin, df in dfs.items():
        if df.shape != shape:
            print(f"Shape mismatch for {coin}. Expected {shape}, got {df.shape}")
            continue
        elif (df.iloc[0, :]["tic"], df.iloc[-1, :]["tic"]) != start_end:
            print(f"Start/End mismatch for {coin}. Expected {start_end}, got {df.iloc[0, :]['tic'], df.iloc[-1, :]['tic']}")
            continue
        else:
            valid_dfs[coin] = df

    return valid_dfs


valid_dfs = filter_dfs(dfs, major_shape, major_se)

Shape mismatch for ENA. Expected (381, 12), got (136, 12)
Shape mismatch for BOME. Expected (381, 12), got (243, 12)
Shape mismatch for TNSR. Expected (381, 12), got (99, 12)
Shape mismatch for MERL. Expected (381, 12), got (10, 12)


In [7]:
df = valid_dfs["SOL"]

df.head()

,datetime,start_time,end_time,interval,number_of_trades,close,high,low,open,volume,tic,toc
symbol,,,,,,,,,,,,
SOL,2024-02-21-16:00:00,2024-02-21-16:00:00,2024-02-21-19:59:59,4h,7129,102.13,104.55,100.27,103.93,147300.90,1708531200000,1708545599999
SOL,2024-02-21-20:00:00,2024-02-21-20:00:00,2024-02-21-23:59:59,4h,4380,105.14,105.82,101.47,102.12,127435.57,1708545600000,1708559999999
SOL,2024-02-22-00:00:00,2024-02-22-00:00:00,2024-02-22-03:59:59,4h,3921,103.66,105.17,102.15,105.17,101283.97,1708560000000,1708574399999
SOL,2024-02-22-04:00:00,2024-02-22-04:00:00,2024-02-22-07:59:59,4h,3486,105.13,105.49,103.20,103.65,127158.41,1708574400000,1708588799999
SOL,2024-02-22-08:00:00,2024-02-22-08:00:00,2024-02-22-11:59:59,4h,3542,105.93,107.15,104.91,105.16,134789.17,1708588800000,1708603199999


In [8]:
_df = pd.concat(valid_dfs.values())
_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21336 entries, BTC to AI
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   datetime          21336 non-null  object 
 1   start_time        21336 non-null  object 
 2   end_time          21336 non-null  object 
 3   interval          21336 non-null  object 
 4   number_of_trades  21336 non-null  int64  
 5   close             21336 non-null  float64
 6   high              21336 non-null  float64
 7   low               21336 non-null  float64
 8   open              21336 non-null  float64
 9   volume            21336 non-null  float64
 10  tic               21336 non-null  int64  
 11  toc               21336 non-null  int64  
dtypes: float64(5), int64(3), object(4)
memory usage: 2.1+ MB


In [9]:
_df.head()

,datetime,start_time,end_time,interval,number_of_trades,close,high,low,open,volume,tic,toc
symbol,,,,,,,,,,,,
BTC,2024-02-21-16:00:00,2024-02-21-16:00:00,2024-02-21-19:59:59,4h,32541,51052.0,51436.0,50638.0,51160.0,978.55598,1708531200000,1708545599999
BTC,2024-02-21-20:00:00,2024-02-21-20:00:00,2024-02-21-23:59:59,4h,33938,51909.0,52000.0,50862.0,51052.0,1273.72087,1708545600000,1708559999999
BTC,2024-02-22-00:00:00,2024-02-22-00:00:00,2024-02-22-03:59:59,4h,25035,51595.0,51911.0,51161.0,51910.0,743.73055,1708560000000,1708574399999
BTC,2024-02-22-04:00:00,2024-02-22-04:00:00,2024-02-22-07:59:59,4h,18721,51866.0,51961.0,51365.0,51595.0,618.39497,1708574400000,1708588799999
BTC,2024-02-22-08:00:00,2024-02-22-08:00:00,2024-02-22-11:59:59,4h,16534,51621.0,52054.0,51520.0,51867.0,709.76714,1708588800000,1708603199999


In [10]:
df = _df

In [11]:
df.to_csv(DATA_DIR / "all.csv", index=True)